In [14]:
import json
from globals import *
import pandas as pd
import numpy as np
from dataprep.dataset import load_reformated_csv, create_dataset
from sklearn.preprocessing import MinMaxScaler

In [20]:
def get_dataset(df,
                variable='B:VIMIN',
                train_split_size: float = 0.7):

    dataset = df[variable].values
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (-1, 1))

    train_size = int(len(dataset) * train_split_size)
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

    X_train, Y_train = create_dataset(train,
                                      look_back =  LOOK_BACK,
                                      look_forward = LOOK_FORWARD)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    Y_train = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1]))

    return  X_train, Y_train

In [23]:
# Load data to initialize the env
with open(DATA_CONFIG) as json_file:
    data_config = json.load(json_file)
data = load_reformated_csv(filename = data_config['data_dir'] + data_config['data_filename'],
                           nrows = NSTEPS)
data['B:VIMIN'] = data['B:VIMIN'].shift(-1)
data = data.set_index(pd.to_datetime(data.time))
data = data.dropna()
data = data.drop_duplicates()
# logger.info ( 'Number of samples:{}'.format ( data.shape ) )
data_list = []
x_train = []
# get_dataset also normalizes the data
for v in range(len(VARIABLES)):
    data_list.append(get_dataset(data, variable=VARIABLES[v]))
#     scalers.append(data_list[v][0])
    x_train.append(data_list[v][0])
# Axis
concate_axis = 1
X_train = np.concatenate(x_train, axis=concate_axis)

In [24]:
X_train.shape

(174982, 6, 15)

In [6]:
import os
from datetime import datetime
now = datetime.now ( )
timestamp = now.strftime ( "D%m%d%Y-T%H%M%S" )
print ( "date and time:" , timestamp )

parent_dir = "./results/policy_plots/"
ep_dir = "episode_plots_{}".format(timestamp)
dir_path = os.path.dirname(parent_dir)
path = os.path.join(dir_path, ep_dir)
os.mkdir(path)

date and time: D02012022-T205421


In [7]:
print(path)

./results/policy_plots/episode_plots_D02012022-T205421
